In [7]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

from tensorflow.keras.layers import Dense
from tensorflow.keras import models
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from IPython.display import display

In [2]:
input_dir="../input/efficientnet_output_straight/"
dataset_dir="../input/ranzcr-clip-catheter-line-classification/"

input_df=pd.read_csv("./models/efficientnet_output.csv")
display(input_df)

,StudyInstanceUID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,-0.112231,0.245513,-0.033396,-0.191451,-0.199094,-0.004325,-0.083669,0.191719,-0.123753,...,0.190715,-0.178776,0.320377,-0.041465,-0.231193,-0.159202,0.118353,-0.183021,0.112336,0.497603
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0.067357,0.011641,0.100913,-0.112439,0.140193,0.442245,-0.244318,0.214603,0.095342,...,0.089758,-0.004185,-0.029220,-0.230071,-0.154650,0.121530,-0.066817,-0.139586,-0.190869,-0.071105
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,-0.073804,0.213021,-0.020444,-0.081371,-0.053769,0.206744,-0.057491,0.169386,0.163935,...,0.290390,0.100370,0.369674,-0.121252,-0.178570,-0.113657,0.314377,0.027548,-0.106127,0.388178
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0.209653,0.341568,0.198626,-0.082370,-0.162807,0.436283,0.111789,-0.146370,0.390531,...,0.268752,-0.081703,0.260819,-0.153018,-0.242180,0.036024,0.245464,0.005475,0.015372,0.091531
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0.084045,0.073493,0.073640,0.048036,0.380339,0.306694,-0.001131,-0.050514,0.304453,...,0.190559,0.103791,-0.089837,-0.000235,-0.185620,0.005118,0.807094,-0.141003,0.348858,-0.144865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,-0.127748,0.166860,0.271803,-0.191275,-0.115660,-0.027046,-0.074921,-0.101646,0.231501,...,0.358037,0.117128,0.165411,-0.111959,-0.222058,0.152683,0.445822,-0.135504,0.062997,-0.000440
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0.010990,0.207189,0.036729,-0.177184,0.057512,0.264610,-0.100801,-0.029718,0.151412,...,0.491667,-0.096260,0.097591,-0.132159,-0.219154,-0.097472,0.493421,-0.214767,-0.058083,0.003562
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0.022863,0.288057,0.463321,-0.051057,-0.071321,0.100492,-0.152650,0.104869,0.151835,...,0.540294,0.009805,0.027360,-0.041865,-0.085383,0.013754,0.266392,-0.097326,-0.098183,0.096486
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,-0.048304,0.354992,0.283024,0.001114,-0.114220,0.141220,0.100976,-0.058647,0.230701,...,0.314984,-0.173124,0.259807,-0.170161,-0.201301,0.382029,0.301187,-0.002982,0.183414,0.186215


In [12]:
input_df.max(axis=1)

0        1.256895
1        1.029967
2        1.112047
3        1.707101
4        1.140373
           ...   
30078    1.247145
30079    1.258481
30080    1.445978
30081    1.285173
30082    1.138580
Length: 30083, dtype: float64

In [10]:
input_shape=(2560,)
encoding_dim=100

model=models.Sequential([
    Dense(encoding_dim*4,activation="relu",input_shape=input_shape),
    Dense(encoding_dim,activation="relu",),
    Dense(encoding_dim*4,activation="relu"),    
    Dense(input_shape[0],activation="sigmoid")
])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 400)               1024400   
_________________________________________________________________
dense_14 (Dense)             (None, 200)               80200     
_________________________________________________________________
dense_15 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_16 (Dense)             (None, 200)               20200     
_________________________________________________________________
dense_17 (Dense)             (None, 400)               80400     
_________________________________________________________________
dense_18 (Dense)             (None, 2560)              1026560   
Total params: 2,251,860
Trainable params: 2,251,860
Non-trainable params: 0
____________________________________________

In [11]:
X_train, X_test=train_test_split(input_df.iloc[:,1:])

adam=Adam(learning_rate=0.0001)
model.compile(optimizer=adam,loss="mse")

fit_callbacks=[callbacks.EarlyStopping(monitor="val_loss",patience=20,mode="min")]

model.fit(X_train,X_train,epochs=200,shuffle=True,validation_data=(X_test,X_test),callbacks=fit_callbacks)

score=model.evaluate(X_test,X_test,verbose=0)
print(f"\nscore: {score}")

Epoch 1/100
115/706 [===>..........................] - ETA: 15s - loss: 0.0934

KeyboardInterrupt: 